In [ ]:
!pip install pydantic

In [ ]:
!pip install langchain_openai

In [ ]:
!pip install langchain_google_genai

In [5]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [6]:
# 1. Using Pydantic BaseModel

class NepalPackageTour(BaseModel):

    """Information about a Nepal Package Tour"""

    name: str = Field(description="Title of the Package")
    description: str = Field(description="Description of the Package")
    price: float = Field(description="Price of the Package")
    duration: int = Field(description="Duration of the Package")
    location: str = Field(description="Location of the Package")
    features: list[str] = Field(description="Features of the Package")

structured_llm_Package = llm.with_structured_output(NepalPackageTour)
result_Package = structured_llm_Package.invoke("Give Me a Package to visit in nepal in this summer")
print("Package Info:\n", result_Package)

Package Info:
 name='Nepal Summer Package' description='This package includes all the major tourist destinations in Nepal.' price=1000.0 duration=7 location='Nepal' features=['Kathmandu sightseeing', 'Pokhara sightseeing', 'Chitwan National Park safari']


In [7]:
# 2. Using TypedDict with Annotated

from typing_extensions import Annotated, TypedDict
from typing import Optional

class Joke(TypedDict):
    """Joke to tell user."""

    setup: Annotated[str, ..., "The setup of the joke"]

    # Alternatively, we could have specified setup as:

    # setup: str                    # no default, no description
    # setup: Annotated[str, ...]    # no default, no description
    # setup: Annotated[str, "foo"]  # default, no description

    punchline: Annotated[str, ..., "The punchline of the joke"]
    # rating: Annotated[int, ..., "How funny the joke is, from 1 to 10"]
    rating: Annotated[Optional[int], None, "How funny the joke is, from 1 to 10"]


structured_llm = llm.with_structured_output(Joke)

structured_llm.invoke("Tell me a joke about cats")

{'setup': 'Why are cats such bad dancers?',
 'punchline': 'Why are cats such bad dancers? Because they have two left feet!'}

In [8]:
# 3. Using JSON Schema

json_schema = {
    "title": "joke",
    "description": "Joke to tell user.",
    "type": "object",
    "properties": {
        "setup": {
            "type": "string",
            "description": "The setup of the joke",
        },
        "punchline": {
            "type": "string",
            "description": "The punchline to the joke",
        },
        "rating": {
            "type": "integer",
            "description": "How funny the joke is, from 1 to 10",
            "default": None,
        },
    },
    "required": ["setup", "punchline", "rating"],
}

structured_llm_joke_json = llm.with_structured_output(json_schema)
result_joke_json = structured_llm_joke_json.invoke("Tell me a joke about cats")
print("\nJoke (JSON Schema):\n", result_joke_json)

Key 'parameters' is not supported in schema, ignoring



Joke (JSON Schema):
 {'rating': 7.0, 'setup': 'Why are cats such bad dancers?', 'punchline': 'Why are cats such bad dancers? Because they have two left feet!'}
